In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot
import os
from copy import deepcopy


from time import time

from math import ceil
from scipy.stats import spearmanr, gamma, poisson
import scipy.sparse as sp
from scipy.io import mmread

from anndata import AnnData, read_h5ad
import scanpy as sc
from scanpy import read
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from torch import tensor
from torch.cuda import is_available
import importlib
import sys
import umap



/work/vanaja_lab/satya/envs/scipennenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-23 19:42:48.312736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745451768.428482   10701 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745451768.450901   10701 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745451768.625819   10701 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 

In [7]:
# Add the src directory to the Python path
sys.path.append(os.path.abspath('/work/vanaja_lab/satya/DEEPOMAPNET/DONET/'))

# Import the sciPENN_API module
import DONET.DONET_API
import DONET.Preprocessing

# Reload the module to ensure the latest changes are reflected
# importlib.reload(SCIPENN.sciPENN_API)

from DONET.DONET_API import DONET_API
from DONET.Preprocessing import preprocess


In [11]:
from DONET.Utils import build_dir
from DONET.Preprocessing import preprocess
from DONET.Data_Infrastructure.DataLoader_Constructor import build_dataloaders
from DONET.Network.DONET import DONET_Model
from DONET.Network.Losses import cross_entropy, sse_quantile, no_loss

In [12]:
GSM_Controls_RNA = sc.read_h5ad('/work/vanaja_lab/satya/R/SatyaPackrat/GSMControlRNA.h5ad')
GSM_Controls_ADT = sc.read_h5ad('/work/vanaja_lab/satya/R/SatyaPackrat/ControlADT.h5ad')


In [13]:
GSM_AML_RNA_A = sc.read_h5ad('/work/vanaja_lab/satya/R/SatyaPackrat/AMLARNA.h5ad')
GSM_AML_ADT_A = sc.read_h5ad('/work/vanaja_lab/satya/R/SatyaPackrat/AMLAADT.h5ad')

In [14]:
GSM_AML_RNA_B = sc.read_h5ad('/work/vanaja_lab/satya/R/SatyaPackrat/AMLBRNA.h5ad')
GSM_AML_ADT_B = sc.read_h5ad('/work/vanaja_lab/satya/R/SatyaPackrat/AMLBADT.h5ad')

In [15]:
import anndata

# Concatenate gene data
adata_gene = anndata.concat(
    [GSM_AML_RNA_B, GSM_AML_RNA_A, GSM_Controls_RNA],
    join='outer',
    label='source',
    keys=['GSM_AML_RNA_B', 'GSM_AML_RNA_A', 'GSM_Controls_RNA']
)

# Concatenate protein data
adata_protein = anndata.concat(
    [GSM_Controls_ADT, GSM_AML_ADT_A, GSM_AML_ADT_B],
    join='outer',
    label='source',
    keys=['GSM_Controls_ADT', 'GSM_AML_ADT_A', 'GSM_AML_ADT_B']
)

# Inspect sample IDs in the gene data (adjust the key if needed)
print("All sample IDs in gene data:", adata_gene.obs['samples'].unique())

# Define your training samples exactly as they appear
train_samples = [
    'AML0612', 'AML3133', 'AML2910', 'AML3050',
    'AML073', 'AML055', 'AML048', 'AML2123', 'AML1371',
    'AML4897', 'AML3948', 'AML0160', 'AML0310',
    'AML0361', 'AML043', 'AML028', 'AML006',
    'AML012', 'AML005', 'AML0048', 'AML022', 'AML0024', 'AML026',
    'AML0114', 'Control4', 'Control2', 'Control3', 'Control5',
    'Control0004', 'Control0058', 'Control0082', 'Control4003'
]

# Build a boolean mask for training cells based on 'samples'
train_mask = adata_gene.obs['samples'].isin(train_samples)
assert train_mask.sum() > 0, f"No cells matched {train_samples}! Check your sample names."

# Subset the gene and protein AnnData objects for training and test sets
adata_gene_train = adata_gene[train_mask].copy()
adata_protein_train = adata_protein[train_mask].copy()
adata_gene_test = adata_gene[~train_mask].copy()
adata_protein_test = adata_protein[~train_mask].copy()

# For both training and test sets, align cell IDs by taking the intersection and sorting
def align_obs(gene_data, protein_data):
    common_cells = sorted(set(gene_data.obs_names).intersection(set(protein_data.obs_names)))
    gene_data_aligned = gene_data[common_cells].copy()
    protein_data_aligned = protein_data[common_cells].copy()
    assert all(gene_data_aligned.obs_names == protein_data_aligned.obs_names), "Cell IDs do not match after alignment!"
    return gene_data_aligned, protein_data_aligned

adata_gene_train, adata_protein_train = align_obs(adata_gene_train, adata_protein_train)
adata_gene_test, adata_protein_test = align_obs(adata_gene_test, adata_protein_test)

print("Train cells:", adata_gene_train.n_obs, "| Test cells:", adata_gene_test.n_obs)




All sample IDs in gene data: ['AML0612' 'AML3762' 'AML3133' 'AML2910' 'AML3050' 'AML2451' 'AML056'
 'AML073' 'AML055' 'AML048' 'AML052' 'AML2123' 'AML1371' 'AML4340'
 'AML4897' 'AML051' 'AML0693' 'AML3948' 'AML3730' 'AML0160' 'AML0310'
 'AML0361' 'AML038' 'AML008' 'AML043' 'AML028' 'AML006' 'AML025' 'AML003'
 'AML012' 'AML005' 'AML0048' 'AML022' 'AML0024' 'AML009' 'AML026' 'AML001'
 'AML0114' 'Control4' 'Control2' 'Control1' 'Control3' 'Control5'
 'Control0004' 'Control0058' 'Control0082' 'Control4003' 'Control0005']
Train cells: 90211 | Test cells: 20486


In [16]:
doublet_bool = (GSM_Controls_RNA.obs['Cell_type_identity'] != 'Doublet')
adata_gene = GSM_Controls_RNA[doublet_bool].copy()
adata_protein = GSM_Controls_ADT[doublet_bool].copy()

In [17]:
# Convert data matrices to float32 if needed
adata_gene_train.X = adata_gene_train.X.astype('float32')
adata_protein_train.X = adata_protein_train.X.astype('float32')
adata_gene_test.X = adata_gene_test.X.astype('float32')
adata_protein_test.X = adata_protein_test.X.astype('float32')

In [18]:
# Define the required parameters
gene_trainsets =  [adata_gene_train] # Your gene training datasets
protein_trainsets = [adata_protein_train]  # Your protein training datasets
gene_test = adata_gene_test  # Optional gene test datasets
gene_list = []  # Your list of genes
train_batchkeys = ['samples']
test_batchkey = 'samples'
select_hvg = True
type_key = 'aml'
cell_normalize = True
log_normalize = True
gene_normalize = True
min_cells = 3
min_genes = 20
batch_size = 128
val_split = 0.1
use_gpu = True

# Create an instance of the sciPENN_API class
DONET = DONET_API(
    gene_trainsets=[adata_gene_train], 
    protein_trainsets=[adata_protein_train], 
    gene_test=adata_gene_test,
    gene_list=[],
    train_batchkeys = ['samples'],
    test_batchkey = 'samples',
    select_hvg=True,
    type_key= 'aml', 
    cell_normalize=True,
    log_normalize=True,
    gene_normalize=True,
    min_cells=3,
    min_genes=20,
    batch_size=128,
    val_split=0.1,
    use_gpu=True
)



Searching for GPU
GPU not detected, falling back to CPU

QC Filtering Training Cells
QC Filtering Testing Cells

QC Filtering Training Genes
QC Filtering Testing Genes

Normalizing Training Cells
Normalizing Testing Cells

Log-Normalizing Training Data
Log-Normalizing Testing Data

Finding HVGs


/work/vanaja_lab/satya/DeepOMAPNet/DONET/Preprocessing.py:137: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  tmp = gene_train.concatenate(gene_test, batch_key=None).copy()



Normalizing Gene Training Data by Batch


100%|██████████| 30/30 [00:02<00:00, 14.28it/s]



Normalizing Protein Training Data by Batch


100%|██████████| 30/30 [00:01<00:00, 25.79it/s]



Normalizing Gene Testing Data by Batch


100%|██████████| 12/12 [00:00<00:00, 24.29it/s]


In [ ]:
# Calculate top expressing surface marker proteins and create UMAP plots

# First, let's compute UMAP embedding for the protein data
print("Computing UMAP embedding for protein data...")
sc.tl.umap(adata_protein_train, random_state=42)

# Calculate top expressing surface marker proteins
print("Calculating top expressing surface marker proteins...")

# Get the top 10 most highly expressed proteins (surface markers)
# We'll use mean expression across all cells as the metric
protein_expression_means = np.array(adata_protein_train.X.mean(axis=0)).flatten()
protein_names = adata_protein_train.var_names

# Get indices of top 10 most expressed proteins
top_10_indices = np.argsort(protein_expression_means)[-10:][::-1]  # Top 10, sorted descending
top_10_proteins = protein_names[top_10_indices]
top_10_expression = protein_expression_means[top_10_indices]

print("Top 10 expressing surface marker proteins:")
for i, (protein, expr) in enumerate(zip(top_10_proteins, top_10_expression)):
    print(f"{i+1:2d}. {protein}: {expr:.4f}")

# Create UMAP plots for the top 10 surface marker proteins
print("\nCreating UMAP plots for top 10 surface marker proteins...")

# Set up the plotting parameters
sc.settings.set_figure_params(dpi=80, facecolor='white')

# Create a figure with subplots for the top 10 proteins
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

# Plot each of the top 10 proteins
for i, protein in enumerate(top_10_proteins):
    ax = axes[i]
    
    # Create the UMAP plot for this protein
    sc.pl.umap(adata_protein_train, 
               color=protein, 
               ax=ax, 
               show=False,
               title=f'{protein}',
               colorbar_loc='right',
               frameon=False)
    
    # Customize the plot
    ax.set_title(f'{protein}', fontsize=12, fontweight='bold')
    ax.set_xlabel('UMAP 1', fontsize=10)
    ax.set_ylabel('UMAP 2', fontsize=10)

# Remove any unused subplots
for i in range(len(top_10_proteins), len(axes)):
    fig.delaxes(axes[i])

plt.suptitle('UMAP Plots of Top 10 Expressing Surface Marker Proteins', 
             fontsize=16, fontweight='bold', y=0.95)
plt.tight_layout()
plt.show()

# Also create individual plots for better visualization
print("\nCreating individual UMAP plots for detailed view...")

# Create individual plots for the top 5 proteins for better detail
fig, axes = plt.subplots(1, 5, figsize=(25, 5))

for i, protein in enumerate(top_10_proteins[:5]):
    ax = axes[i]
    
    # Create the UMAP plot for this protein
    sc.pl.umap(adata_protein_train, 
               color=protein, 
               ax=ax, 
               show=False,
               title=f'{protein}',
               colorbar_loc='right',
               frameon=False,
               size=20)
    
    # Customize the plot
    ax.set_title(f'{protein}', fontsize=14, fontweight='bold')
    ax.set_xlabel('UMAP 1', fontsize=12)
    ax.set_ylabel('UMAP 2', fontsize=12)

plt.suptitle('Detailed UMAP Plots of Top 5 Expressing Surface Marker Proteins', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nSummary:")
print(f"Total number of proteins: {adata_protein_train.n_vars}")
print(f"Total number of cells: {adata_protein_train.n_obs}")
print(f"UMAP embedding computed with random_state=42")
print(f"Top expressing protein: {top_10_proteins[0]} (mean expression: {top_10_expression[0]:.4f})")
print(f"Lowest of top 10: {top_10_proteins[-1]} (mean expression: {top_10_expression[-1]:.4f})")
